In [1]:
#Les importations
import numpy as np
import random
import matplotlib.pyplot as plt

from scipy.stats import multivariate_normal
from sklearn.mixture import GaussianMixture
from sklearn.datasets import make_blobs
from scipy.stats.kde import gaussian_kde

In [2]:
#Un bout de code pour représenter graphiquement une gaussienne : pas utile
mu = 0
sigma=1
nums=[]
for i in range(10000):  
    temp = random.gauss(mu, sigma)  
    nums.append(temp)  
        
# plotting a graph  
#plt.hist(nums, bins = 200)  
#plt.show() 

In [3]:
#La génération des paramètres du générateur de pommes
ptot=0 #le dénominateur, somme des 3 probabilités
mu=[0,0,0] #les 3 moyennes, entre 0 et 10
sigma=[0,0,0] #les 3 écarts types entre 0 et 10
p=[0,0,0] #la probabilité de chaque arbre
for i in range(3):
  mu[i]=random.random()*10
  sigma[i]=random.random()*10
  p[i]=random.random()
  ptot+=p[i]
for i in range(3):
  p[i]=p[i]/ptot
for i in range(3):
  print(mu[i],sigma[i],p[i])

2.86044061142026 3.341988165541099 0.1492680525443975
0.5334259993582302 0.025763168421943927 0.20889186053729172
3.6932152534803118 1.186521259193677 0.6418400869183107


In [4]:
#Le générateur de pommes
def generepomme(mu,sigma,p):
  r=random.random()
  if r<p[0]:
    arbre=0
  elif r<p[0]+p[1]:
    arbre=1
  else: arbre=2 #on a sélectionné l'arbre
  pos=random.gauss(mu[arbre], sigma[arbre]) #on applique l'arbre
  #print(arbre,pos)
  return pos
#On teste
nums=[]
for i in range(1000):
  x=generepomme(mu,sigma,p)
  nums.append(x)
plt.hist(nums, bins = 200)  
plt.show() 

In [5]:
def baseline(tirages,nbtests):
  return(random.random()*5,5+random.random()*5)
  #cet algorithme est stupide et fonctionne assez mal! Dans ce cas il n'y a pas de paramètre mais on peut utiliser en particulier l'historique des pommes précédentes !

In [6]:
def dernier_vu(tirages,nb_tests):#une heuristique: mettre le panier là où la dernière pomme est tombée
  if nb_tests>1:
    x=tirages[nb_tests-1]
    y=tirages[nb_tests-2]
  elif nb_tests>0:
    x=tirages[nb_tests-1]
    y=0.3
  else:
    x=0.7
    y=0.3
  return(x,y)

In [7]:
"""
Uniquement en 2D
"""
def EMGMM2D(tirages, resultats):
    
    ######################################################
    # Partie à update en 2D
    if len(tirages) > 2:
        X = np.reshape(tirages, (-1,1))
        Y = resultats
    else:
        X = np.reshape([0.3, 0.7], (-1, 1))
        Y = 0, 0

    X = [np.append(element, [1.0]) for element in X]
    #Y = np.reshape(Y, (-1, 1))
    print(X)
    print(Y)
    ########################################################
    X = np.dot(X,np.random.RandomState(0).randn(2,2))

    x,y = np.meshgrid(np.sort(X[:,0]),np.sort(X[:,1]))
    XY = np.array([x.flatten(),y.flatten()]).T

    GMM = GaussianMixture(n_components=2).fit(X) 


    prediction = GMM.predict_proba(Y)
    print(prediction)

In [18]:
"""
Beaucoup trop lent (10sec exec/3600) et inefficace (30/100)
"""
def kde_gaussienne(tirages):
    if len(tirages) < 2:
        data = [4,6]
    else: 
        data = tirages

    kde = gaussian_kde(data)

    x = np.linspace(-15, 20.0, 50)
    y = [kde(i) for i in x]


In [19]:
#Protocole de test de la solution
nb_tests=0
nb_corrects=0
tirages=[]
resultats = []
for i in range(3600):
    #x,y=baseline(tirages,nb_tests)#ici remplacer baseline par le nom de la fonction à tester
    x,y=dernier_vu(tirages,nb_tests)
    #EMGMM(tirages, resultats)
    
    #kde_gaussienne(tirages)
    z=generepomme(mu,sigma,p)
    #print('x',x,'y',y,'z',z, nb_corrects,nb_tests)
    tirages.append(z)
    if abs(x-z)<0.5 or abs(y-z)<0.5: #diamètre du seau=1, donc rayon=0.5
        nb_corrects+=1
        resultats.append(1)
    else:
        resultats.append(0)
    nb_tests+=1
print('Nombre de pommes ramassées, sur 100 tombées :',100*nb_corrects/nb_tests)
  

Nombre de pommes ramassées, sur 100 tombées : 29.944444444444443


In [66]:
"""
Tentative de GMM1D
"""
np.random.seed(0)

X = np.linspace(-5,5,num=20)
X0 = X*np.random.rand(len(X))+10 
X1 = X*np.random.rand(len(X))-10 
X2 = X*np.random.rand(len(X)) 
X_tot = np.stack((X0,X1,X2)).flatten()


r = np.zeros((len(X_tot),3))  


gauss_1 = norm(loc=-5,scale=5) 
gauss_2 = norm(loc=8,scale=3)
gauss_3 = norm(loc=1.5,scale=1)

for c,g in zip(range(3),[gauss_1,gauss_2,gauss_3]):
    r[:,c] = g.pdf(X_tot)

for i in range(len(r)):
    r[i] = r[i]/np.sum(r,axis=1)[i] 

print(r)
print(np.sum(r,axis=1)) 


Dimensionality = (60, 3)
[[2.97644006e-02 9.70235407e-01 1.91912550e-07]
 [3.85713024e-02 9.61426220e-01 2.47747304e-06]
 [2.44002651e-02 9.75599713e-01 2.16252823e-08]
 [1.86909096e-02 9.81309090e-01 8.07574590e-10]
 [1.37640773e-02 9.86235923e-01 9.93606589e-12]
 [1.58674083e-02 9.84132592e-01 8.42447356e-11]
 [1.14191259e-02 9.88580874e-01 4.48947365e-13]
 [1.34349421e-02 9.86565058e-01 6.78305927e-12]
 [1.11995848e-02 9.88800415e-01 3.18533028e-13]
 [8.57645259e-03 9.91423547e-01 1.74498648e-15]
 [7.64696969e-03 9.92353030e-01 1.33051021e-16]
 [7.10275112e-03 9.92897249e-01 2.22285146e-17]
 [6.36154765e-03 9.93638452e-01 1.22221112e-18]
 [4.82376290e-03 9.95176237e-01 1.55549544e-22]
 [7.75866904e-03 9.92241331e-01 1.86665135e-16]
 [7.52759691e-03 9.92472403e-01 9.17205413e-17]
 [8.04550643e-03 9.91954494e-01 4.28205323e-16]
 [3.51864573e-03 9.96481354e-01 9.60903037e-30]
 [3.42631418e-03 9.96573686e-01 1.06921949e-30]
 [3.14390460e-03 9.96856095e-01 3.91217273e-35]
 [1.00000000e+0

In [68]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')
import numpy as np
from scipy.stats import norm
np.random.seed(0)

X = np.linspace(-5,5,num=20)
X0 = X*np.random.rand(len(X))+10 # Create data cluster 1
X1 = X*np.random.rand(len(X))-10 # Create data cluster 2
X2 = X*np.random.rand(len(X)) # Create data cluster 3
X_tot = np.stack((X0,X1,X2)).flatten() # Combine the clusters to get the random datapoints from above

r = np.zeros((len(X_tot),3))  


gauss_1 = norm(loc=-5,scale=5) 
gauss_2 = norm(loc=8,scale=3)
gauss_3 = norm(loc=1.5,scale=1)

m = np.array([1/3,1/3,1/3]) # We expect to have three clusters 

pi = m/np.sum(m)

for c,g,p in zip(range(3),[gauss_1,gauss_2,gauss_3],pi):
    r[:,c] = p*g.pdf(X_tot) 
    
for i in range(len(r)):
    r[i] = r[i]/(np.sum(pi)*np.sum(r,axis=1)[i])

m_c = []
for c in range(len(r[0])):
    m = np.sum(r[:,c])
    m_c.append(m) # For each cluster c, calculate the m_c and add it to the list m_c
    

pi_c = []
for m in m_c:
    pi_c.append(m/np.sum(m_c)) # For each cluster c, calculate the fraction of points pi_c which belongs to cluster c


mu_c = np.sum(X_tot.reshape(len(X_tot),1)*r,axis=0)/m_c

var_c = []

for c in range(len(r[0])):
    var_c.append((1/m_c[c])*np.dot(((np.array(r[:,c]).reshape(60,1))*(X_tot.reshape(len(X_tot),1)-mu_c[c])).T,(X_tot.reshape(len(X_tot),1)-mu_c[c])))

gauss_1 = norm(loc=mu_c[0],scale=var_c[0]) 
gauss_2 = norm(loc=mu_c[1],scale=var_c[1])
gauss_3 = norm(loc=mu_c[2],scale=var_c[2])



*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB o

*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB o

*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB o